In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [ ]:
def count_files(directory):
    return sum([len(files) for _, _, files in os.walk(directory)])

train_data_dir = '/content/drive/MyDrive/AI_Enabled_Car_Parking/CarParking_Dataset/train_data/train'
validation_data_dir = '/content/drive/MyDrive/AI_Enabled_Car_Parking/CarParking_Dataset/train_data/test'

files_train = count_files(train_data_dir)
files_validation = count_files(validation_data_dir)

print(files_train, files_validation)


381 164


In [ ]:
# Model parameters
img_width, img_height = 128, 128
train_data_dir = "/content/drive/MyDrive/AI_Enabled_Car_Parking/CarParking_Dataset/train_data/train"
validation_data_dir = "/content/drive/MyDrive/AI_Enabled_Car_Parking/CarParking_Dataset/train_data/test"
nb_train_samples = files_train
nb_validation_samples = files_validation
batch_size = 32
epochs = 5

In [ ]:
# Loading pre-trained VGG16 model
base_model = applications.VGG16(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))

# Freezing layers in the base model
for layer in base_model.layers[:10]:
    layer.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Adding custom dense layer for binary classification
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(units=2, activation='softmax')(x)
model = Model(base_model.input, x)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [ ]:
# Compiling the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    fill_mode="nearest",
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=5)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    fill_mode="nearest",
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=5)


In [ ]:
# Creating data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical")

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    class_mode="categorical")

Found 381 images belonging to 2 classes.
Found 164 images belonging to 2 classes.


In [ ]:
# Model training
checkpoint = ModelCheckpoint("car1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

history_object = model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[checkpoint, early])

<ipython-input-11-c9c9e40c8e87>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_object = model.fit_generator(


Epoch 1/5
12/12 [==============================] - ETA: 0s - loss: 0.4304 - accuracy: 0.7402 

12/12 [==============================] - 233s 20s/step - loss: 0.4304 - accuracy: 0.7402 - val_loss: 0.2763 - val_accuracy: 0.8780
Epoch 2/5
12/12 [==============================] - ETA: 0s - loss: 0.1864 - accuracy: 0.9475 

12/12 [==============================] - 168s 14s/step - loss: 0.1864 - accuracy: 0.9475 - val_loss: 0.7644 - val_accuracy: 0.8476
Epoch 3/5
12/12 [==============================] - ETA: 0s - loss: 0.1705 - accuracy: 0.9475 

12/12 [==============================] - 178s 15s/step - loss: 0.1705 - accuracy: 0.9475 - val_loss: 0.2663 - val_accuracy: 0.9268
Epoch 4/5
12/12 [==============================] - ETA: 0s - loss: 0.0890 - accuracy: 0.9685 

12/12 [==============================] - 177s 15s/step - loss: 0.0890 - accuracy: 0.9685 - val_loss: 0.2307 - val_accuracy: 0.9207
Epoch 5/5
12/12 [==============================] - ETA: 0s - loss: 0.0781 - accuracy: 0.9816 

12/12 [==============================] - 167s 14s/step - loss: 0.0781 - accuracy: 0.9816 - val_loss: 0.3581 - val_accuracy: 0.8902


In [ ]:
# Model prediction function
def prediction(path):
    img = tf.keras.preprocessing.image.load_img(path, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescaling
    pred = np.argmax(model.predict(img_array))
    print(f"The image belongs to class {pred}")

In [ ]:
# Example prediction
image_path = "/content/drive/MyDrive/AI_Enabled_Car_Parking/CarParking_Dataset/test_images/scene1410.jpg"
prediction(image_path)

1/1 [==============================] - 1s 550ms/step
The image belongs to class 1


In [ ]:
# Save the model as a pickle file
import pickle

model_dict = {'model': model.to_json(), 'weights': model.get_weights()}

with open('/content/drive/MyDrive/AI_Enabled_Car_Parking/spot_dict.pickle', 'wb') as f:
    pickle.dump(model_dict, f)